In [10]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import json
import os
import datetime
import random

scores = "data.json"

class JsonDB:
    def __init__(self, filename):  # Fixed constructor
        self.filename = filename
        if os.path.exists(filename):
            with open(filename, "r") as f:
                try:
                    self.data = json.load(f)
                except json.JSONDecodeError:
                    self.data = {}
        else:
            self.data = {}

        if "users" not in self.data:
            self.data["users"] = {}

    def save(self):
        with open(self.filename, "w") as f:
            json.dump(self.data, f, indent=4)

    def get_user(self, username):
        return self.data["users"].get(username)

    def add_user(self, username, numbers):
        self.data["users"][username] = {
            "numbers": numbers,
            "swaps": []
        }
        self.save()

    def update_numbers(self, username, numbers):
        if username in self.data["users"]:
            self.data["users"][username]["numbers"] = numbers
            self.save()

    def log_swap(self, username, swap_record):
        if username in self.data["users"]:
            self.data["users"][username]["swaps"].append(swap_record)
            self.save()

    def reset_user(self, username, numbers):
        if username in self.data["users"]:
            self.data["users"][username] = {
                "numbers": numbers,
                "swaps": []
            }
            self.save()

class SortingApp:
    def __init__(self, master):  # Fixed constructor
        self.master = master
        self.master.title("Sorting App with JSON Database")
        self.master.configure(bg="black")

        self.db = JsonDB(scores)

        self.username = simpledialog.askstring("User Login", "Enter your name:")
        if not self.username:
            self.username = "Guest"

        self.numbers = self.generate_unique_list()
        self.db.add_user(self.username, self.numbers)

        self.selected_index = None

        self.label = tk.Label(master, text=f"Welcome, {self.username}!", font=("Arial", 14), fg="white", bg="black")
        self.label.pack(pady=10)

        self.list_frame = tk.Frame(master, bg="black")
        self.list_frame.pack(pady=10)

        self.status_label = tk.Label(master, text="Click two numbers to swap.", fg="white", bg="black")
        self.status_label.pack(pady=5)

        self.control_frame = tk.Frame(master, bg="black")
        self.control_frame.pack(pady=10)

        self.update_list_buttons()
        self.add_controls()

    def generate_unique_list(self, size=7, min_val=1, max_val=99):
        return random.sample(range(min_val, max_val), size)

    def update_list_buttons(self):
        for widget in self.list_frame.winfo_children():
            widget.destroy()
        for idx, value in enumerate(self.numbers):
            btn = tk.Button(self.list_frame, text=str(value), width=6, height=2,
                            command=lambda i=idx: self.select_index(i),
                            bg="gray20", fg="white", activebackground="gray40", activeforeground="white")
            btn.grid(row=0, column=idx, padx=5)

    def select_index(self, index):
        if self.selected_index is None:
            self.selected_index = index
            self.status_label.config(text=f"Selected index {index}. Select another to swap.")
        else:
            swap_record = {
                "swap_move": f"Swap index {self.selected_index} with index {index}",
                "timestamp": datetime.datetime.now().isoformat()
            }

            self.numbers[self.selected_index], self.numbers[index] = self.numbers[index], self.numbers[self.selected_index]
            self.db.log_swap(self.username, swap_record)
            self.db.update_numbers(self.username, self.numbers)

            self.selected_index = None
            self.update_list_buttons()
            self.status_label.config(text="Swapped!")

    def add_controls(self):
        btn_style = {"width": 15, "bg": "gray20", "fg": "white", "activebackground": "gray40", "activeforeground": "white"}

        tk.Button(self.control_frame, text="Find Max", command=self.find_max, **btn_style).grid(row=0, column=0, padx=5, pady=5)
        tk.Button(self.control_frame, text="Find Min", command=self.find_min, **btn_style).grid(row=0, column=1, padx=5, pady=5)
        tk.Button(self.control_frame, text="Linear Search", command=self.linear_search, **btn_style).grid(row=0, column=2, padx=5, pady=5)
        tk.Button(self.control_frame, text="Find Index", command=self.find_index, **btn_style).grid(row=0, column=3, padx=5, pady=5)
        tk.Button(self.control_frame, text="Leaderboard", command=self.show_leaderboard, **btn_style).grid(row=0, column=4, padx=5, pady=5)

        tk.Button(self.control_frame, text="Auto Sort", command=self.auto_sort, width=20, bg="green", fg="white",
                  activebackground="darkgreen", activeforeground="white").grid(row=1, column=1, columnspan=1, pady=10)

        tk.Button(self.control_frame, text="Shuffle", command=self.shuffle_list, width=20, bg="darkorange", fg="white",
                  activebackground="orangered", activeforeground="white").grid(row=1, column=2, columnspan=1, pady=10)

    def shuffle_list(self):
        self.numbers = self.generate_unique_list()
        self.db.reset_user(self.username, self.numbers)
        self.selected_index = None
        self.update_list_buttons()
        self.status_label.config(text="List shuffled and swaps reset!")

    def find_max(self):
        max_val = max(self.numbers)
        messagebox.showinfo("Maximum", f"The maximum number is {max_val}")

    def find_min(self):
        min_val = min(self.numbers)
        messagebox.showinfo("Minimum", f"The minimum number is {min_val}")

    def linear_search(self):
        target = simpledialog.askinteger("Search", "Enter number to search:")
        if target is None:
            return
        for i, val in enumerate(self.numbers):
            if val == target:
                messagebox.showinfo("Found", f"{target} found at index {i}")
                return
        messagebox.showinfo("Not Found", f"{target} is not in the list.")

    def find_index(self):
        target = simpledialog.askinteger("Find Index", "Enter number to find index of:")
        if target is None:
            return
        if target in self.numbers:
            idx = self.numbers.index(target)
            messagebox.showinfo("Index", f"Index of {target} is {idx}")
        else:
            messagebox.showinfo("Not Found", f"{target} not found.")

    def auto_sort(self):
        swap_record = {
            "swap_move": "Auto sorted the entire list",
            "timestamp": datetime.datetime.now().isoformat()
        }

        self.numbers.sort()
        self.db.log_swap(self.username, swap_record)
        self.db.update_numbers(self.username, self.numbers)

        self.update_list_buttons()
        self.status_label.config(text="List auto-sorted!")

    def show_leaderboard(self):
        users = self.db.data["users"]
        leaderboard_list = []
        for username, info in users.items():
            moves_count = len(info.get("swaps", []))
            leaderboard_list.append((username, moves_count))
        leaderboard_list.sort(key=lambda x: x[1])

        lb_window = tk.Toplevel(self.master)
        lb_window.title("Leaderboard - Optimal Moves")
        lb_window.configure(bg="black")

        tk.Label(lb_window, text="SCORES",
                 font=("Arial", 14, "bold"), fg="white", bg="black").pack(pady=10)

        for rank, (user, moves) in enumerate(leaderboard_list, start=1):
            text = f"{rank}. {user} - {moves} move{'s' if moves != 1 else ''}"
            tk.Label(lb_window, text=text, font=("Arial", 12), fg="white", bg="black").pack(anchor="w", padx=20)

        lb_window.geometry("300x300+{}+{}".format(
            self.master.winfo_x() + 100,
            self.master.winfo_y() + 100
        ))

# Correct entry point
if __name__ == "__main__":
    root = tk.Tk()

    window_width = 700
    window_height = 300
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_cordinate = int((screen_width / 2) - (window_width / 2))
    y_cordinate = int((screen_height / 2) - (window_height / 2))
    root.geometry(f"{window_width}x{window_height}+{x_cordinate}+{y_cordinate}")

    root.configure(bg="black")
    root.resizable(False, False)

    app = SortingApp(root)
    root.mainloop()
